<a href="https://colab.research.google.com/github/aimldlnlp/bi2dec-using-rnn/blob/main/bi2dec_conversion_using_rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Setup

In [31]:
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import cifar10
import numpy as np
import matplotlib.pyplot as plt

### Data Generation and Preprocessing

In [32]:
def bin2int(bin_list):
  bin_str = ""
  for k in bin_list:
    bin_str += str(int(k))
  return int(bin_str,2)

def dataset(num, bin_len):
  x = np.zeros((num, bin_len))
  y = np.zeros((num))

  for i in range(num):
    x[i] = np.round(np.random.rand(bin_len)).astype(int)
    y[i] = bin2int(x[i])

  return x, y

### Model Definition

In [33]:
x_len = 8
no_of_samples = 1000
lr = 0.01
training_steps= 50000
display_step = 5000

n_input = x_len
timestep = 1
n_hidden = 16
n_output = 1

trainX, trainY = dataset(no_of_samples, x_len)
testX, testY = dataset(20, x_len)

tf.compat.v1.disable_eager_execution()

X = tf.compat.v1.placeholder(tf.float32, [None, timestep, n_input])
Y = tf.compat.v1.placeholder(tf.float32, [None, n_output])

weights = tf.Variable(tf.random.normal([n_hidden, n_output]))
bias = tf.Variable(tf.random.normal([n_output]))

In [34]:
def RNN(x, weights, bias):
  x = tf.reshape(x, [-1, timestep, n_input])
  lstm_layer = layers.LSTM(n_hidden)
  outputs = lstm_layer(x)
  return tf.matmul(outputs, weights) + bias

### Training and Evaluation

In [41]:
logits = RNN(X, weights, bias)

trainX = np.reshape(trainX, [-1, timestep, n_input])
trainY = np.reshape(trainY, [-1, n_output])

testX = np.reshape(testX, [-1, timestep, n_input])
testY = np.reshape(testY, [-1, n_output])

loss = tf.reduce_mean(tf.square(logits - Y))
optimizer = tf.compat.v1.train.RMSPropOptimizer(learning_rate=lr)
train = optimizer.minimize(loss)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [50]:
with tf.compat.v1.Session() as sess:
  sess.run(tf.compat.v1.global_variables_initializer())
  for step in range(training_steps):
    _, _loss = sess.run([train, loss], feed_dict={X: trainX, Y: trainY})
    if (step + 1) % display_step == 0:
      print("Step:", '%04d' % (step + 1), "loss=", "{:.9f}".format(_loss))

    # print("Optimization Finished!")

  result = sess.run(logits, feed_dict={X: testX})
  result = sess.run(tf.round(result))

  print("Real \t Guess")
  for i in range(20):
    if testY[i] == result[i]:
      print("True")
    else:
      print("False")
      print(testY[i], ' -> ', result[i])

Step: 5000 loss= 0.504598498
Step: 10000 loss= 0.105137564
Step: 15000 loss= 0.079905272
Step: 20000 loss= 0.071498051
Step: 25000 loss= 0.088098630
Step: 30000 loss= 0.055011015
Step: 35000 loss= 0.097597003
Step: 40000 loss= 0.108191825
Step: 45000 loss= 0.043241143
Step: 50000 loss= 0.049219634
Real 	 Guess
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
